In [9]:
!pip install langchain_community
!pip install replicate

In [20]:
import pandas as pd
import os
import random
from langchain_community.llms import Replicate
import os
from google.colab import userdata

In [21]:
# Set the API token
api_token = userdata.get('api_token')
os.environ["REPLICATE_API_TOKEN"] = api_token

# Model setup
model = "ibm-granite/granite-3.3-8b-instruct"
output = Replicate(
    model=model,
    replicate_api_token=api_token,
    )

In [22]:
# Dataset Loading
DATASET_PATH = '/content/train.csv'
df = pd.read_csv(DATASET_PATH)

print(f"Dataset loaded successfully with {len(df)} rows.")
print("Columns available:", df.columns.tolist())

Dataset loaded successfully with 18612 rows.
Columns available: ['instruction', 'input', 'output', 'prompt']


In [23]:
# --- Select a Code Snippet to Explain ---
# Select a random row from dataset for demo
selected_row = df.sample(n=1).iloc[0]

problem_instruction = selected_row['instruction']
problem_input = selected_row['input']
code_to_explain = selected_row['output']
original_generation_prompt = selected_row['prompt']

print(f"\n--- Selected Problem Instruction: ---\n{problem_instruction}")
print("_"*50)
print(f"\n--- Corresponding Input: ---\n{problem_input}")
print("_"*50)
print(f"\n--- Original Code Generation Prompt: ---\n{original_generation_prompt}")
print("_"*50)
print(f"\n--- Code to be Explained: ---\n```python\n{code_to_explain}\n```")
print("_"*50)


--- Selected Problem Instruction: ---
Create a python code to convert a string of comma-separated numbers into list.
__________________________________________________

--- Corresponding Input: ---
numbers = '1,2,3,4,5,6'
__________________________________________________

--- Original Code Generation Prompt: ---
Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Create a python code to convert a string of comma-separated numbers into list.

### Input:
numbers = '1,2,3,4,5,6'

### Output:
numbers = '1,2,3,4,5,6'

numbers_list = [int(x) for x in numbers.split(',')]

print(numbers_list) # [1, 2, 3, 4, 5, 6]
__________________________________________________

--- Code to be Explained: ---
```python
numbers = '1,2,3,4,5,6'

numbers_list = [int(x) for x in numbers.split(',')]

print(numbers_list) # [1, 2, 3, 4, 5, 6]
```
__________________________________________________


In [24]:
# --- Crafting the Prompt for Granite ---
# Now, you can incorporate the instruction and input into your prompt for better context.
prompt = f"""You are an expert programmer and code explainer.
Please explain the following Python code snippet in simple, clear language.
The code was generated to fulfill the following instruction:
"{problem_instruction}"

 It was designed to handle input like:
"{problem_input}"

Focus on:
1.  **Purpose:** What does the code aim to achieve, considering the instruction?
2.  **How it Works:** Step-by-step breakdown of its functionality.
3.  **Key Concepts/Libraries:** Mention any important programming concepts or external libraries used.

Code:
```python
{code_to_explain}"""

explanation = output.invoke(prompt)
print("\n--- Code Explanation (from Granite) ---")
print(explanation)


--- Code Explanation (from Granite) ---
1. Purpose:
The purpose of this Python code snippet is to convert a string of comma-separated numbers into a list of integers. This fulfills the instruction provided, which required creating a Python code to handle input like "1,2,3,4,5,6" and convert it into a list format suitable for numerical operations.

2. How it Works:
- First, the code assigns the string '1,2,3,4,5,6' to the variable `numbers`.
- The line `numbers.split(',')` splits this string into a list of substrings using the comma (`,`) as a delimiter. The result is `['1', '2', '3', '4', '5', '6']`.
- The list comprehension `[int(x) for x in numbers.split(',')]` then iterates over each element (`x`) in the list obtained from the previous step. For each `x`, it applies the `int()` function to convert the string representation of a number into an integer.
- As a result, a new list `numbers_list` is created containing the integer values `[1, 2, 3, 4, 5, 6]`.
- Finally, the `print(number